# Invasion experiment of $2+1$ under fast migration

In [ ]:
# Importing libraries
import matplotlib.pyplot as pl
pl.rcParams['hatch.linewidth'] = 2
hfont = {'fontname': 'Helvetica'}

import numpy as np
import pandas as pd
from scipy.integrate import solve_ivp


def Simulation_MC_with_ancestor(t, f, r1, r2, gamma, qA, p2A):

    x1 = f[0] # Single cells from multicellular strategy
    x2 = f[1] # Two-cell groups from multicellular strategy
    y  = f[2] # Unicellular ancestor

    qB = 1 - qA
    p2B = 1 - p2A

    dy = r1 * y - gamma * y * ((qA**2 + qB**2) * (y + x1) + 2 * (p2A * qA + p2B * qB) * x2)
    dx1 = 2 * r2 * x2 - r1 * x1 - gamma * x1 * ((qA**2 + qB**2) * (x1 + y) + 2 * (p2A * qA + p2B * qB) * x2)
    dx2 = r1 * x1 - gamma * x2 * ((p2A * qA + p2B * qB) * (y + x1) + 2 * (p2A**2 + p2B**2) * x2)
    return [dx1, dx2, dy]

gamma = 1
r1A = 15
sigma = 1 / 3  # Determines how environment's B growth rate compares to A growth rate
r1B = sigma * r1A

points = np.linspace(0, 1, 1000)  # Values of qA and p2A to explore

# Invasion simulation controls
t0 = 0.0
tf = 1e6              
eps = 1e-2            # initial mutant introduction
extinct_tol = 1e-7   # threshold to call extinct


# tau = cost of multicellularity; r2 = tau * r1
for tau in [0.8, 0.94, 0.98]:
    print("tau="+str(tau))

    # Parameters that depend on tau
    r2A = tau * r1A
    r2B = tau * r1B

    # Saves the ecological outcome:
    # 0 = unicellular ancestor dominates (R2>C2, no simulation needed)
    # 1 = coexistence (ancestor + multicell persist)
    # 2 = multicell dominates (ancestor displaced)
    Eco_O = np.zeros((len(points), len(points)), dtype=int)

    for iq, qA in enumerate(points):
        qB = 1 - qA
        
        if iq % 100 == 0:
            print(iq)

        for ip, p2A in enumerate(points):
            p2B = 1 - p2A

            # Environment-averaged growth rates
            r1 = r1A * qA + r1B * qB
            r2 = r2A * p2A + r2B * p2B

            R2 = r2 / r1
            C2 = (p2A * qA + p2B * qB) / (qA**2 + qB**2)

            if R2 > C2: #If Multicellularity can invade:

                # Ancestor-only equilibrium:
                y_star = r1 / (gamma * (qA**2 + qB**2))

                # Introduce mutant at very low density
                f0 = [eps, 0.0, y_star]  # [x1, x2, y]

                #Simulate
                sol = solve_ivp(
                    Simulation_MC_with_ancestor,
                    (t0, tf),
                    f0,
                    args=(r1, r2, gamma, qA, p2A),
                    method="LSODA"
                )

                x1_f, x2_f, y_f = sol.y[:, -1]
                mutant_total = x1_f + 2 * x2_f

                ancestor_alive = y_f > extinct_tol
                mutant_alive = mutant_total > extinct_tol

                if ancestor_alive and mutant_alive:
                    Eco_O[iq, ip] = 1  # coexistence
                elif mutant_alive and not ancestor_alive:
                    Eco_O[iq, ip] = 2  # multicell dominates
                else:
                    Eco_O[iq, ip] = -1 # unexpected outcome

    df_Eco_O = pd.DataFrame(Eco_O, index=points, columns=points)
    df_Eco_O.to_csv(f"Data/Figure 3 and S1/Fast_migration_Invasion_tau{tau}.csv", index_label="qA/p2A")


tau=0.8
0
100
200
300
400
500
600
700
800
900
tau=0.94
0
100
200
300
400
500
600
700
800
900
tau=0.98
0
100
200
300
400
500
600
700
800
900


## Alternative method
Multicellular dominance: when the mutant can invade but a unicellular mutant cannot invade.
The latter can be obtained by evaluating the invasion condition from the unicellular ancestor's equation:
$$\frac{r_{1,A}}{\gamma} > (q_A^2 + q_B^2)\cdot x^*_1 + 2\cdot (p_{2,A}q_A + p_{2,B}q_B)\cdot x^*_2$$
where $x_1^*$ and $x_2^*$ are the equilibrium densities of the multicellular $2+1$ life cycle in isolation (obtained numerically below). The resultshere are identical to the slower method above.

In [ ]:
# Importing libraries 
import matplotlib.pyplot as pl
pl.rcParams['hatch.linewidth'] = 2
hfont = {'fontname':'Helvetica'}

import numpy as np
from numpy import random
import pandas as pd
import seaborn as sns
from scipy.integrate import solve_ivp
from numpy import linalg as LA


#Model equations for a multicellular mutant in isolation
def Simulation_MC(t,f,r1,r2,gamma,qA,p2A):

    #variables
    x1 = f[0] #Single cells
    x2 = f[1] #two-cell groups
    qB=1-qA
    p2B=1-p2A


    dx1 = 2*r2*x2 - r1*x1 - gamma*x1*((qA**2 + qB**2)*(x1)+2*(p2A*qA + p2B*qB)*x2)
    
    dx2= r1*x1 -gamma*x2*((p2A*qA + p2B*qB)*(x1)+2*(p2A**2 + p2B**2)*x2)
    
    return[dx1,dx2]
    

#tau = cost of multicellularity; r2= tau * r1    
for tau in [0.8,0.94,0.98]:
    
    #Parameters
    gamma=1
    r1A=15
    sigma=1/3 #Determines how environment's B growth rate compares to A growth rate 
    r1B= sigma*r1A
    r2A=tau*r1A
    r2B=tau*r1B


    points=np.linspace(0,1,1000) #Values of p1A (qA) and p2A to explore
    Eco_O=np.zeros((len(points),len(points))) #Saves the ecological outcome (i.e., coexistence or dominance)
    
    
    for iq in range(len(points)):
        for ip in range(len(points)):
            
            qA=points[iq] #p1A = qA
            p2A=points[ip]
            
            qB=1-qA 
            p2B=1-p2A

            r1=r1A*qA + r1B*qB
            r2=r2A*p2A + r2B*p2B
            
            R2 = r2/r1
            C2 = (p2A*qA +p2B*qB)/(qA**2 + qB**2)

            if R2>C2: #If multicellularity evolves

                #Ecological outcome:
                t0=0
                tf=10000000

                q0 = [r1/gamma,r2/gamma]
                #parameters
                #calls integrator
                xsol = solve_ivp(Simulation_MC,(t0,tf),q0,args=(r1,r2,gamma,qA,p2A), dense_output=True,method='LSODA')
                xf= xsol.sol(tf)
                x1, x2 = xf

                if r1/gamma > (qA**2 + qB**2)*x1 + 2*(p2A*qA + p2B*qB)*x2: #Unicellular ancestor can invade the 2+1 multicellular life cycle
                    Eco_O[iq][ip]=1 #Coexistence (=1)
                else:
                    Eco_O[iq][ip]=2 #Dominance (=2)
        
    df_Eco_O = pd.DataFrame(Eco_O, index=points, columns=points)
    #df_Eco_O.to_csv(f"Data/Figure 3 and S1/Fast_migration_Invasion_tau{tau}.csv", index_label="qA/p2A")